<h1>Анализ формулы один с использованием методов машинного обучения и математической статистики</h1>

<h2>Первоначальной нашей целью будет прогноз вероятности совершения обгона на каждом из участков трассы.</h2>

In [1]:
import fastf1 as ff1
import pandas as pd
import numpy as np
import datetime
import math

ff1.Cache.enable_cache('cache')
session = ff1.get_session(2021, 22, 'R')
quali = ff1.get_session(2021, 22, 'Q')
quali = pd.DataFrame(quali.results)

In [2]:
laps = session.load_laps(with_telemetry=True)
session.load_telemetry()

core           INFO 	Loading laps for Abu Dhabi Grand Prix - Race [v2.1.13]
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for session_status_data
utils          INFO 	NumExpr defaulting to 8 threads.
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Loaded data for 19 drivers: ['55', '7', '31', '33', '18', '77', '22', '14', '16', '44', '6', '63', '99', '3', '47', '11', '10', '4', '5']
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data


In [3]:
laps

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,SpeedST,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
0,0 days 01:03:54.885000,33,NaT,1,1,0 days 00:23:01.492000,NaT,NaT,0 days 00:00:38.428000,0 days 00:00:33.563000,...,318.0,SOFT,4.0,False,0 days 01:02:21.871000,Red Bull,VER,1,False,2021-12-12 13:03:21.881
1,0 days 01:05:23.988000,33,0 days 00:01:29.103000,2,1,NaT,NaT,0 days 00:00:18.116000,0 days 00:00:38.026000,0 days 00:00:32.961000,...,302.0,SOFT,5.0,False,0 days 01:03:54.885000,Red Bull,VER,1,True,2021-12-12 13:04:54.895
2,0 days 01:06:52.815000,33,0 days 00:01:28.827000,3,1,NaT,NaT,0 days 00:00:17.999000,0 days 00:00:38.043000,0 days 00:00:32.785000,...,301.0,SOFT,6.0,False,0 days 01:05:23.988000,Red Bull,VER,1,True,2021-12-12 13:06:23.998
3,0 days 01:08:21.841000,33,0 days 00:01:29.026000,4,1,NaT,NaT,0 days 00:00:18.019000,0 days 00:00:38.125000,0 days 00:00:32.882000,...,303.0,SOFT,7.0,False,0 days 01:06:52.815000,Red Bull,VER,1,True,2021-12-12 13:07:52.825
4,0 days 01:09:50.559000,33,0 days 00:01:28.718000,5,1,NaT,NaT,0 days 00:00:17.999000,0 days 00:00:38.083000,0 days 00:00:32.636000,...,303.0,SOFT,8.0,False,0 days 01:08:21.841000,Red Bull,VER,1,True,2021-12-12 13:09:21.851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,0 days 02:24:55.281000,47,0 days 00:02:11.605000,53,3,0 days 02:23:04.752000,NaT,0 days 00:00:40.395000,0 days 00:00:48.541000,0 days 00:00:42.669000,...,203.0,SOFT,4.0,False,0 days 02:22:43.676000,Haas F1 Team,MSC,4,False,2021-12-12 14:23:43.686
994,0 days 02:26:39.361000,47,0 days 00:01:44.080000,54,3,NaT,NaT,0 days 00:00:19.591000,0 days 00:00:42.890000,0 days 00:00:41.599000,...,268.0,SOFT,5.0,False,0 days 02:24:55.281000,Haas F1 Team,MSC,4,False,2021-12-12 14:25:55.291
995,0 days 02:28:57.976000,47,0 days 00:02:18.615000,55,3,NaT,NaT,0 days 00:00:22.400000,0 days 00:01:02.169000,0 days 00:00:54.046000,...,257.0,SOFT,6.0,False,0 days 02:26:39.361000,Haas F1 Team,MSC,4,False,2021-12-12 14:27:39.371
996,0 days 02:31:17.001000,47,0 days 00:02:19.025000,56,3,NaT,NaT,0 days 00:00:28.870000,0 days 00:00:56.316000,0 days 00:00:53.839000,...,258.0,SOFT,7.0,False,0 days 02:28:57.976000,Haas F1 Team,MSC,4,False,2021-12-12 14:29:57.986


Сделаем подготовку данных для просчета обгонов
Отберем для каждого гонщика те записи, где менялся гонщик впереди, аналогичные же записи отберем для других гонщиков

In [28]:
def get_ticks_to_process(laps_to_process):
    driver_numbers =  laps_to_process['DriverNumber'].unique()
    ticks_to_process = []
    for driver in driver_numbers:
        telemetry = laps_to_process.pick_driver(driver).get_telemetry()
        data = telemetry['DriverAhead'].to_frame()
        data['DriverAhead'] = pd.to_numeric(data['DriverAhead'])
        data = data.fillna(0)
        data = data.diff(axis=0)
        telemetry['DriverAhead_difference'] = data['DriverAhead']
        telemetry.dropna(subset = ['DriverAhead_difference'], inplace=True)
        telemetry = telemetry[(telemetry['DriverAhead_difference'] != 0)]
        ticks_to_process += list(telemetry['Time'].unique())
    return list(sorted(list(dict.fromkeys(ticks_to_process))))

In [78]:
# взять текущий тик -> ближайший тик гонщика впереди -> ближайший тик его гонщика и т.д. ...
def sort_not_calculated(drivers_to_process, laps_to_process, last_driver_in_peleton, current_time):
    drivers_dict = {}
    drivers_to_delete = []
    result = []

    for driver in drivers_to_process:
        driver_ahead = laps_to_process.pick_driver(driver).get_telemetry()
        ahead_tick = driver_ahead[(driver_ahead['Time'] >= current_time)].index.min()
        ahead_tick = driver_ahead.loc[ahead_tick]
        if len(ahead_tick) == 0:
            drivers_to_delete.append(driver)
            break
        driver_ahead_num = ahead_tick['DriverAhead']
        try:
            _ = drivers_dict[driver_ahead_num]
            return [], drivers_to_delete, True
        except Exception as _:
            drivers_dict[driver_ahead_num] = driver

    for c in range(len(drivers_dict)):
        try:
            last_driver_in_peleton = drivers_dict[last_driver_in_peleton]
            result.append(last_driver_in_peleton)
        except Exception as _:
            return [], drivers_to_delete, True

    return result, drivers_to_delete, False


def get_peleton_by_tick(current_time, driver_laps_to_process, drivers_peleton_to_process, laps_to_process):
    current_time_index = driver_laps_to_process[driver_laps_to_process['Time'] > current_time].index.min()
    tick = driver_laps_to_process.loc[[current_time_index]].iloc[0]
    current_time = tick['Time']
    print(current_time)
    driver_ahead_num = tick['DriverAhead']
    current_peleton = [int(drivers_peleton_to_process[0]), int(tick['DriverAhead'])]
    error_counter = 0
    max_possible_num_of_errors = 0
    while driver_ahead_num != '':
        driver_ahead = laps.pick_driver(driver_ahead_num).get_telemetry()
        ahead_tick = driver_ahead[(driver_ahead['Time'] >= current_time)].index.min()
        ahead_tick = driver_ahead.loc[ahead_tick]
        driver_ahead_num = ahead_tick['DriverAhead']

        if error_counter > max_possible_num_of_errors:
            not_calculated_drivers = list(set(drivers_peleton_to_process).difference(current_peleton))
            print(not_calculated_drivers, driver_ahead_num, current_peleton, 'too much errors')
            return current_peleton, not_calculated_drivers, True, current_time # последний параметр отвечает за успешность обработки tick'а

        if driver_ahead_num != '':
            driver_ahead_num = int(driver_ahead_num)
            if driver_ahead_num not in current_peleton:
                current_peleton.append(driver_ahead_num)
            elif driver_ahead_num == current_peleton[-2]:
                current_peleton = current_peleton[:-2] + [current_peleton[-1], current_peleton[-2]]
                driver_ahead = laps_to_process.pick_driver(driver_ahead_num).get_telemetry()
                current_time_index = driver_ahead[(driver_ahead['Time'] > current_time) & (driver_ahead['DriverAhead'] != '')].index.min()
                current_time = driver_ahead.loc[[current_time_index]].iloc[0]['Time']
            else:
                driver_ahead_num = current_peleton[0]
                current_peleton = current_peleton[:1]
                current_time_index = driver_laps_to_process[driver_laps_to_process['Time'] > current_time].index.min()
                tick = driver_laps_to_process.loc[[current_time_index + 1]].iloc[0]
                current_time = tick['Time']
                max_possible_num_of_errors = 2
                error_counter += 1
                print('tick updated')
        else:
            not_calculated_drivers = list(set(drivers_peleton_to_process).difference(current_peleton))
            updated_tail, drivers_to_delete, error = sort_not_calculated(not_calculated_drivers, laps_to_process, current_peleton[0], current_time)
            current_peleton = updated_tail + current_peleton
            for driver in drivers_to_delete:
                drivers_peleton_to_process.remove(driver)
            if error:
                driver_ahead_num = not_calculated_drivers[0]
                current_peleton = not_calculated_drivers[:1]
                current_time_index = driver_laps_to_process[driver_laps_to_process['Time'] > current_time].index.min()
                tick = driver_laps_to_process.loc[[current_time_index + 1]].iloc[0]
                current_time = tick['Time']
                max_possible_num_of_errors = max(2, len(not_calculated_drivers))
                error_counter += 1
                print('tick updated from tail')
            else:
                return current_peleton, not_calculated_drivers, False, current_time # последний параметр отвечает за успешность обработки tick'а

In [48]:
drivers_peleton_all = list(map(int, reversed(quali['number'].tolist())))
drivers_peleton = []
for i in range(len(drivers_peleton_all)):
    try:
        _ = laps.pick_driver(drivers_peleton_all[i]).get_telemetry()
        drivers_peleton.append(drivers_peleton_all[i])
    except Exception as _:
        pass

overtakes_to_process = get_ticks_to_process(laps)

In [ ]:
print(datetime.datetime.now())
success_ticks = 0
failed_ticks = 0
peleton = drivers_peleton[:1]
last_time = 0
drivers_peleton_updates = {'Peleton': [], 'Not_proceeded': [], 'Time_min': [], 'Time_max': []}
for overtake_tick in overtakes_to_process:
    if overtake_tick > last_time:
        driver_laps = laps.pick_driver(peleton[0]).get_telemetry()
        peleton, not_processed, failed, last_time = get_peleton_by_tick(overtake_tick, driver_laps, drivers_peleton, laps)
        if not failed:
            success_ticks += 1
            drivers_peleton_updates['Peleton'].append(peleton)
            drivers_peleton_updates['Not_proceeded'].append(not_processed)
            drivers_peleton_updates['Time_min'].append([overtake_tick])
            drivers_peleton_updates['Time_max'].append([overtake_tick])
            print('success', peleton, not_processed)
        else:
            failed_ticks += 1
            print('failed', peleton, not_processed)
print(success_ticks / (success_ticks + failed_ticks))

2022-03-10 01:40:31.315311
0 days 00:00:00.681000
tick updated
tick updated from tail
success [5, 47, 18, 33, 22, 14, 10, 11, 55, 63, 99, 3, 4, 77, 31, 7, 16, 6, 44] []
0 days 00:00:02.661000
tick updated from tail
tick updated from tail
success [47, 5, 33, 18, 10, 22, 14, 63, 11, 55, 99, 31, 4, 77, 3, 16, 6, 7, 44] []
0 days 00:00:03.321000
success [47, 5, 33, 10, 18, 14, 22, 63, 11, 55, 31, 99, 4, 77, 16, 3, 7, 6, 44] []
0 days 00:00:03.541000
success [47, 5, 33, 18, 10, 14, 22, 11, 63, 55, 31, 4, 99, 77, 3, 16, 7, 6, 44] []
0 days 00:00:03.761000
success [47, 5, 33, 14, 18, 10, 22, 11, 63, 55, 31, 4, 99, 77, 16, 3, 6, 7, 44] []
0 days 00:00:03.981000
success [47, 5, 33, 14, 10, 18, 22, 11, 31, 63, 55, 4, 99, 77, 16, 3, 6, 7, 44] []
0 days 00:00:04.201000
success [47, 5, 33, 14, 18, 10, 22, 11, 31, 63, 55, 4, 99, 77, 16, 3, 6, 7, 44] []
0 days 00:00:04.421000
success [47, 33, 5, 14, 18, 10, 22, 11, 31, 63, 55, 99, 4, 77, 16, 3, 7, 6, 44] []
0 days 00:00:04.861000
success [47, 33, 5, 

In [ ]:
drivers_peleton_updates_df = pd.DataFrame(drivers_peleton_updates)
drivers_peleton_updates_df.to_csv('overtakes.csv', encoding='utf-8', index=False)

## Посмотрим на визуализацию мест обгонов на одной конкретной трассе за разные года. Для этого выберем трассу которая не менялась с 2014 года.
 <i style="color: #ff2800"> небольшое уточнение, все координаты представлены в 1/10 метра, мы разделим всю трассу на зоны

In [175]:
df = laps.pick_driver(44).get_telemetry()
df[df['Time'] > pd.Timedelta('0 days 00:00:42')]

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
369,2021-12-05 17:34:26.770,0 days 01:02:19.552000,,3.407942,0 days 00:00:42.038000,9058,145,4,64,0,1,car,2192.728333,0.007188,OnTrack,-4355,21782,123
370,2021-12-05 17:34:26.875,0 days 01:02:19.657000,,3.408469,0 days 00:00:42.143000,9244,146,4,66,0,1,pos,2197.014681,0.007202,OnTrack,-4396,21769,123
371,2021-12-05 17:34:27.010,0 days 01:02:19.792000,,3.408997,0 days 00:00:42.278000,9431,148,4,68,0,1,car,2202.595000,0.007220,OnTrack,-4446,21745,122
372,2021-12-05 17:34:27.095,0 days 01:02:19.877000,,3.409524,0 days 00:00:42.363000,9571,150,4,72,0,1,pos,2206.162137,0.007232,OnTrack,-4477,21728,122
373,2021-12-05 17:34:27.250,0 days 01:02:20.032000,,3.410051,0 days 00:00:42.518000,9711,153,4,77,0,1,car,2212.795000,0.007254,OnTrack,-4531,21690,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66112,2021-12-05 19:39:59.709,0 days 03:07:52.491000,,0.230000,0 days 02:06:14.977000,8799,248,8,0,100,0,car,305027.872222,0.999898,OnTrack,-1191,-70,118
66113,2021-12-05 19:39:59.720,0 days 03:07:52.502000,,0.230000,0 days 02:06:14.988000,8711,246,8,0,100,0,pos,305028.624336,0.999900,OnTrack,-1194,-64,118
66114,2021-12-05 19:39:59.939,0 days 03:07:52.721000,,0.230000,0 days 02:06:15.207000,8624,244,8,0,100,0,pos,305043.416787,0.999948,OnTrack,-1238,63,118
66115,2021-12-05 19:39:59.949,0 days 03:07:52.731000,,0.230000,0 days 02:06:15.217000,8537,243,8,0,100,0,car,305044.072222,0.999951,OnTrack,-1240,68,118


In [317]:
x_size_of_sector = 50
y_size_of_sector = 50

In [319]:
overtakes = overtakes.drop_duplicates(subset=['First_driver', 'Second_driver', 'Date'])

<h2> Давайте создадим нашу трассу </h2>

In [320]:
def num_of_overtakes_in_segment(point, overtakes_df):
    return len(overtakes_df[(overtakes_df['X'] == math.floor(point['X'] // 10  // x_size_of_sector)) & (overtakes_df['Y'] == math.floor(point['Y'] // 10 // y_size_of_sector))])

lap = laps.pick_fastest()
tel = lap.get_telemetry()

x = np.array(tel['X'].values)
y = np.array(tel['Y'].values)

overtakes_to_display = pd.DataFrame({"X": x, "Y": y})
overtakes_to_display['nOvertakes'] = overtakes_to_display.apply(lambda i: num_of_overtakes_in_segment(i, overtakes), axis=1)

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
overtakes_to_display.sort_values(by='nOvertakes', ascending=False)

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

fig,ax = plt.subplots(figsize=(30, 30), dpi=200)

overtake = overtakes_to_display['nOvertakes'].to_numpy().astype(float)

cmap = cm.get_cmap('Paired')
lc_comp = LineCollection(segments, norm=plt.Normalize(1, cmap.N+1), cmap=cmap)
lc_comp.set_array(overtake)
lc_comp.set_linewidth(4)
ax.add_collection(lc_comp)

# plt.plot(x, y)
# plt.scatter(overtakes_to_display[overtakes_to_display['nOvertakes'] != 0]['X'], overtakes_to_display[overtakes_to_display['nOvertakes'] != 0]['Y'], color='y', s=20)
# plt.scatter(overtakes['X'] * 10 * x_size_of_sector, overtakes['Y'] * 10 * y_size_of_sector, color='r', s=10)
# plt.axis('equal')

major_ticks = np.arange(-7000, 23000, 500)
minor_ticks = np.arange(-7000, 23000, 500)

ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)
plt.grid()
plt.xticks(rotation=90)
plt.show()